# Option Pricing Logic for all One-Time Trained YFinance Models

Make sure to create the required files first in other notebook(s) before evaluating them here

In [1]:
!pip install fbm
!pip install yfinance==0.1.63

In [2]:
import os
# Set working directory to MarketGenerators folder
# If you are working on LRZ servers, create the folder "MarketGenerators" and then specify something like
path = "/dss/dsshome1/02/YOUR_LRZ_USER_NAME/MarketGenerators"
os.chdir(path)

In [3]:
import numpy as np
from src.evaluate import metrics as m
from src.data.make_dataset import DataLoader
import src.helper.utils as utils

In [ ]:
# Define the base directory
base_dir = "numerical_results_Liao"
# Define the folders of interest
model_folders = ["YFinance"]
# Define the target subfolders
relevant_folders_single_model = ["n-in=0Y"]
target_subfolders = ["CWGAN", "GMMN", "RCGAN", "SigCWGAN", "TimeGAN"]
option_types_K = ["European", "Asian"]
n_days_list = [5, 10, 21, 252]
# Parameters
S0 = 1
t = 0
# Only evaluate lookback option at time T ==> Make grid size large
look_back_grid_size = 1000000
n_batches = 100

In [ ]:
for nDays in n_days_list:
    K_grid = np.linspace(0.95, 1.1, 4) if nDays >= 21 else np.linspace(1, 1.08, 3)
    settings = [f"{metric}_K={K:.2f}" for metric in option_types_K for K in K_grid]
    settings.append("Lookback")
    T = nDays / 252
    # Initialize empty last specification
    last_spec = ""
    last_seed_spec = ""
    last_year_folder = ""

    # Loop through each model folder
    for model_folder in model_folders:
        print(f"Start evaluating all {model_folder}-based models for {nDays} days maturity and {n_batches} batches...")
        model_path = os.path.join(base_dir, model_folder)

        # Traverse the directory tree
        for root, dirs, files in os.walk(model_path):
            gen_model = os.path.basename(root)

            if gen_model in target_subfolders:
                nYearsFolderName = root.split("/")[3]
                if nYearsFolderName not in relevant_folders_single_model:
                    continue
                if root.split("/")[4] != "seed=42":
                    # Only seed 42 is considered (other seeds are retrained models)
                    continue

                # Read the relevant npy files
                generated_file = os.path.join(root, "generated_returns_rescaled.npy")
                input_file = os.path.join(root, "input_returns_unscaled.npy")
                if os.path.exists(generated_file) and os.path.exists(input_file):
                    input_prices_df, generated_prices_df, generated_returns = utils.load_input_and_generated_returns(
                        input_file, generated_file, nDays, T
                    )
                    input_model, model_spec = root.split("/")[1:3]
                    if model_spec != last_spec:
                        # New input model specifications => recalculate input metrics
                        last_spec = model_spec
                        results_call = {setting: {gan: [] for gan in target_subfolders + ["Input"]} for setting in settings}
                        results_put = {setting: {gan: [] for gan in target_subfolders + ["Input"]} for setting in settings}
                        model_desc = "/".join(root.split("/")[1:5])
                        recalculate_input = True
                        print(f"   Evaluating model {model_desc}...")
                        european_engine, asian_engine, lookback_engine = utils.initialize_all_option_engines(
                            input_prices_df, generated_prices_df, T, t=t, S0=S0, input_is_real_data=True
                        )

                        # Get respective path metrics
                        with utils.Capturing([]) as summary_output:
                            european_engine.r = m.print_basic_non_gbm_metrics(
                                n_periods=T,
                                annualization_factor=252,
                                ground_paths_df=input_prices_df,
                                recovered_paths_df=generated_prices_df,
                                approx_df=input_prices_df,
                                return_threshold=0.03
                            )
                            asian_engine.r = european_engine.r
                            lookback_engine.r = european_engine.r
                        recalculate_input = True

                    print(f"         Calculating option prices for {gen_model} paths")
                    for batch in range(n_batches):
                        european_engine.gen_paths_df = generated_prices_df.iloc[batch * 1000:(batch + 1) * 1000, :]
                        asian_engine.gen_paths_df = generated_prices_df.iloc[batch * 1000:(batch + 1) * 1000, :]
                        lookback_engine.gen_paths_df = generated_prices_df.iloc[batch * 1000:(batch + 1) * 1000, :]
                        european_engine.calc_all_K(K_grid, recalculate_input=recalculate_input)
                        asian_engine.calc_all_K(K_grid, recalculate_input=recalculate_input)
                        lookback_engine.calc_all_T(grid_size=look_back_grid_size, recalculate_input=recalculate_input)

                        results_call, results_put = utils.fill_results(
                            european_engine, asian_engine, lookback_engine, results_call, results_put,
                            gen_model, recalculate_input=recalculate_input, has_input_dev=False
                        )
                        recalculate_input = False

                    if gen_model == "GMMN":
                        print("      Saving results...")
                        relevant_dir = "/".join(root.split("/")[:-2])

                        utils.save_stat_analysis_to_csv(
                            settings, results_put, lookback_engine, european_engine, asian_engine,
                            relevant_dir, target_subfolders, nDays, "put", one_time_trained=True
                        )
                        utils.save_stat_analysis_to_csv(
                            settings, results_call, lookback_engine, european_engine, asian_engine,
                            relevant_dir, target_subfolders, nDays, "call", one_time_trained=True
                        )
                else:
                    print("No file found yet.")

print("Done.")